In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
def limit_mem():
    K.get_session().close()
    cfg = K.tf.compat.v1.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.compat.v1.Session(config=cfg))

1 Physical GPUs, 1 Logical GPUs


In [2]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.run_functions_eagerly(False)


In [3]:
#import af libaries
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow_addons as tfa
from PIL import Image
import imagehash
import imageio
import random
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers



In [21]:
#test is not working i need to have a look at the download again
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')
print("Amount of tranning's data: " + str(len(train)))
#print("Amount of test data: " + str(len(test)))
image_size = 100

train['Rank'] = pd.cut(train['Pawpularity'], bins=[0,21,41,61,81, float('Inf')], labels=[0,1,2,3,4])


Amount of tranning's data: 9912


ValueError: invalid literal for int() with base 10: 'Inf'

In [5]:
#Loading images
def get_train_file_path(image_id):
    return r"C:\Users\tobia\Documents\GitHub\PetFinder\train\{}.jpg".format(image_id)

def get_test_file_path(image_id):
    return r"C:\Users\tobia\Documents\GitHub\PetFinder\test\{}.jpg".format(image_id)

train['file_path'] = train['Id'].apply(get_train_file_path)
#test['file_path'] = test['Id'].apply(get_test_file_path)


# Shows the first 5, change the paramter for more
display(train.head())


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,Rank,file_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,3,C:\Users\tobia\Documents\GitHub\PetFinder\trai...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,2,C:\Users\tobia\Documents\GitHub\PetFinder\trai...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,1,C:\Users\tobia\Documents\GitHub\PetFinder\trai...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,0,C:\Users\tobia\Documents\GitHub\PetFinder\trai...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,3,C:\Users\tobia\Documents\GitHub\PetFinder\trai...


In [ ]:
#Visualizing the spread??
train.hist(figsize=(15,15))
plt.tight_layout()
plt.show()

In [ ]:
train['Rank'].hist(figsize=(5,5))
counts = train['Rank'].value_counts()
print(counts)

In [ ]:
##### Mean of Pawpularity remove [] if you want the see mean on everything
print("Mean value of train "+ str(train['Pawpularity'].mean()))
train.info()

In [ ]:
#Kinda slow might be a faster way to do it.
#It gets height and width of each image and then calculate the aspect ratio
widths = []
heights = []
ratios = []
for file_path in tqdm(train['file_path']):
    image = imageio.imread(file_path)
    h, w, _ = image.shape
    heights.append(h)
    widths.append(w)
    ratios.append(w / h)

In [ ]:
#First we plot it to see the values
plt.figure(figsize=(15,8))
plt.title(f'Height and Width', size=24)
plt.hist(heights, bins=32, label='Heights')
plt.hist(widths, bins=32, label='Widths')
plt.legend(prop={'size': 16})
plt.show()

In [ ]:
#https://www.kaggle.com/markwijkhuizen/petfinder-eda-yolov5-obj-detection-tfrecords#Image-EDA
display(pd.Series(ratios).describe())
plt.figure(figsize=(15,8))
plt.title(f'Ratios', size=24)
plt.hist(ratios, bins=15, label='Ratios')
plt.legend(prop={'size': 15})
plt.show()

In [ ]:
#Hard to see the coalition on the ???bar??? graph so we create a scatter point graph
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
points = ax.scatter(widths, heights, color='blue', alpha=0.5, s=ratios, picker=True)
ax.set_title("Image Resolution")
ax.set_xlabel("Width", size=14)
ax.set_ylabel("Height", size=14)

In [ ]:
#Shows 5x5 images and their title is their paw score
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    image = plt.imread(train.iloc[i, -1])
    #image = plt.cvtColor(image, cv2.COLOR_BGR2RGB)
    Pawpularity = train.iloc[i, -3]
    plt.imshow(image)
    plt.title(f"Pawpularity Score: {Pawpularity}")
plt.show()

In [ ]:
#Images with the lowest value
sorted_lowest = (train.sort_values('Pawpularity'))
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    image = plt.imread(sorted_lowest.iloc[i, -1])
    Pawpularity = sorted_lowest.iloc[i, -3]
    plt.imshow(image)
    plt.title(f"Pawpularity Score: {Pawpularity}")
plt.show()

In [ ]:
#Images with the highest value
sorted_highest = (train.sort_values('Pawpularity',ascending=False))
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    image = plt.imread(sorted_highest.iloc[i, -1])
    Pawpularity = sorted_highest.iloc[i, -3]
    plt.imshow(image)
    plt.title(f"Pawpularity Score: {Pawpularity}")
plt.show()

In [ ]:
#ligner der er duplicates som skal fjernes 
#https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/278497 kode til at fjerne dem findes her
tqdm.pandas()
def get_hash(file_path):
    img = Image.open(file_path)
    img_hash = imagehash.phash(img)
    
    return img_hash.hash.reshape(-1).astype(np.uint8)
    
train['phash'] = train['file_path'].progress_apply(get_hash)

In [ ]:
def find_similar_images(threshold=0.90):
    # Number of Duplicate Images Found
    duplicate_counter = 1
    # Indices of Duplicate Images
    duplicate_idxs = set()
    # For each image in the train dataset
    for idx, phash in enumerate(tqdm(train['phash'])):
        # Compute the similarity to all other images
        for idx_other, phash_other in enumerate(train['phash']):
            # Similarity score is imply the percentage of equal bits
            similarity = (phash ==phash_other).mean()
            # Prevent self comparison, threshold similarity and ignore repetetive duplicate detection
            if idx != idx_other and similarity > threshold and not(duplicate_idxs.intersection([idx, idx_other])):
                # Update Duplicate Indices
                duplicate_idxs.update([idx, idx_other])
                # Get DataFrame rows
                row = train.loc[idx]
                row_other = train.loc[idx_other]
                # Plot Duplicate Images
                fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,5))
                ax[0].imshow(imageio.imread(row['file_path']))
                ax[0].set_title(f'Idx: {idx}, Pawpularity: {row["Pawpularity"]}')
                ax[1].imshow(imageio.imread(row_other['file_path']))
                ax[1].set_title(f'Idx: {idx_other}, Pawpularity: {row_other["Pawpularity"]}')
                plt.suptitle(f'{duplicate_counter} | PHASH Similarity: {similarity:.3f}')
                plt.show()
                # Increase Duplicate Counter
                duplicate_counter += 1
                
    # Return Indices of Duplicates
    return duplicate_idxs
    
duplicate_idxs = find_similar_images()
#The same images don't have the same paw score. The majority's paw score's diffrence is around 10.
#But some got over double the score'

In [ ]:
print(f'Found {len(duplicate_idxs)} Duplicate Images')
# Removing Duplicate Images, 
#Code taken from https://www.kaggle.com/markwijkhuizen/petfinder-eda-yolov5-obj-detection-tfrecords#Image-EDA
#IMPORTANT(I think so atleast)! If this code is run twice it will drop both images instead of just one of the duplicates
#Hhich image is correct one? They got diffrent scores
train = train.drop(duplicate_idxs).reset_index(drop=True)
print("Amount of tranning's data: " + str(len(train)))

In [6]:
#We need some more preproccsing and data normalization
def dataNormalization(image):
    image = Image.open(image)
    #tf.image.convert_image_dtype(image, tf.float32)
    #image = tf.reshape(image, [352, 352, 3])
    image = tf.image.resize(image, (image_size, image_size))
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [7]:
#train['file_path'] = train['file_path'].apply(dataNormalization)
#image = plt.imread(train.loc[0,'file_path'])
#ds = tf.data.Dataset.from_tensor_slices(dict(train))
AUTOTUNE = tf.data.experimental.AUTOTUNE

#Split in test and train
train, test = train_test_split(train, test_size=0.2)
print("Amount of tranning's data: " + str(len(train)))
print("Amount of test data: " + str(len(test)))

train_dataset = train['file_path'].map(dataNormalization)
test_dataset = test['file_path'].map(dataNormalization)


Amount of tranning's data: 7929
Amount of test data: 1983


In [8]:
#checks if the values are normalized
#print(train_dataset[0])
#we need proper names here
#train_pred = train.loc[:,'Pawpularity']
#test_pred = test.loc[:,'Pawpularity']


train_pred = train.loc[:,'Rank']
test_pred = test.loc[:,'Rank']


In [23]:
print(train_pred)
#type(pred)
#result = pd.merge(train_dataset, pred, left_on=None)
#train_dataset = pd.merge(train_dataset, train_pred, left_index=True, right_index=True)
#test_dataset = pd.merge(test_dataset, test_pred, left_index=True, right_index=True)

<class 'numpy.ndarray'>


In [ ]:
print(train_dataset)

In [ ]:
display(train_dataset.head())

#print(train_dataset['file_path'])
#display(result.head())

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(5 * 5):
    plt.subplot(5, 5, i+1)
    plt.title('Original image')
    image =  train_dataset.iloc[i]
    plt.imshow(image)
plt.show(block=False)

In [9]:
def augment(augmented):
    # We need to test with the diffrent type of augment to see if they hurt the acc. 
    # We should also take some of the fuctions we used in CNN
    augmented = tf.image.random_brightness(augmented, 0.05)
    augmented = tf.image.random_flip_left_right(augmented)
    augmented = tf.image.random_saturation(augmented, 0.95, 1)
    augmented = tf.image.random_contrast(augmented, 0.95, 1)
    augmented = tf.image.random_hue(augmented, 0.05)
    augmented = shear(augmented)
    return augmented

def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(original)

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)
    
def shear(image):
    #shear
    #de her værdier skal nok ændres i
    random_number_shear = random.uniform(-0.2,0.2)
    shear = random_number_shear
    shear_rotate = shear * -20
    image = tfa.image.transform(image, [1.0, shear, shear_rotate, 0.0, 1.0, 0.0, 0.0, 0.0])
    return image

def crop(image):
    # Add 6 pixels of padding
    image = tf.image.resize_with_crop_or_pad(image, img_height + 6, img_width + 6) 
    # Random crop back to the original size
    image = tf.image.random_crop(image, size=[img_height, img_width, 1])
    return image

In [ ]:
print(train.head())

In [ ]:
#Side by side comparrison with the new ratio
for i in range(4 * 5):
    result_image = train_dataset.iloc[i]
    train_image = plt.imread(train.iloc[i,-2])
    visualize(train_image,result_image)

In [ ]:
for i in range(3):
    image = train_dataset.iloc[i]
    changed_image = augment(image)
    visualize(image,changed_image)

In [ ]:
#Simple CNN
model = tf.keras.models.Sequential([
layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Flatten(),
layers.Dense(units=128, activation='relu'),
layers.Dense(units=1)
])

In [10]:
#CNN
model = tf.keras.models.Sequential([
layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Dropout(0.25),
layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
layers.MaxPool2D(pool_size=2, strides=2),
layers.Dropout(0.25),
layers.Flatten(),
layers.Dense(units=1024, activation='relu'),
layers.Dropout(0.50),
layers.Dense(units=5, activation='softmax')
])



In [11]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer = 'adam', 
              metrics=['accuracy'])

In [12]:
#print(train_dataset.shape)
#print(test_dataset.shape)
#print(train_dataset)
def labelNormalize(value):
    return value/100;
#train_pred = train_pred.map(labelNormalize)
#test_pred = test_pred.map(labelNormalize)

#print(train_dataset)
#test_dataset.dtypes

X = np.array(train_dataset.tolist())
Y = np.array(test_dataset.tolist())

#clf.fit(test_dataset[0].tolist(), labels)
#np.array(test_dataset).astype("float32")


In [13]:
train_pred = np.array(train_pred.tolist())
test_pred = np.array(test_pred.tolist())


In [14]:
temp=pd.DataFrame()

def augmentAfterEpoch():
    temp = train_dataset.map(augment)
    newX = np.array(temp.tolist())
    #navn.append(pd.DataFrame(temp).T)
    #navn.extend([temp])
    return newX
#print(temp)
    #a = torch.tensor(temp)
    #navn2 = torch.cat((a,navn2))

#navn = pd.merge(temp, train_pred, left_index=True, right_index=True)
#print(temp)

In [ ]:
print(type(temp))

In [ ]:
#print(type(train_pred))
#print(train_dataset)
#print(X)
#print(X)


In [15]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('petfinder.keras', save_best_only=True)
]

In [16]:
history = model.fit(X,train_pred,validation_data = (Y,test_pred),shuffle=True,batch_size=16, epochs = 30,callbacks=callbacks)
newX = augmentAfterEpoch()
history2 = model.fit(X,train_pred,validation_data = (Y,test_pred),shuffle=True,batch_size=16, epochs = 30,callbacks=callbacks)
newX = augmentAfterEpoch()
history3 = model.fit(X,train_pred,validation_data = (Y,test_pred),shuffle=True,batch_size=16, epochs = 30,callbacks=callbacks)

#fit(
#    x=None, y=None, batch_size=None, epochs=1, verbose='auto',
#    callbacks=None, validation_split=0.0, validation_data=None, shuffle=True,
#    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
#    validation_steps=None, validation_batch_size=None, validation_freq=1,
#    max_queue_size=10, workers=1, use_multiprocessing=False
#)


Epoch 1/30
496/496 [==============================] - 15s 24ms/step - loss: 1.3209 - accuracy: 0.5138 - val_loss: 1.3223 - val_accuracy: 0.5169
Epoch 2/30
496/496 [==============================] - 11s 21ms/step - loss: 1.3073 - accuracy: 0.5152 - val_loss: 1.3259 - val_accuracy: 0.5169
Epoch 3/30
496/496 [==============================] - 11s 22ms/step - loss: 1.3132 - accuracy: 0.5152 - val_loss: 1.3198 - val_accuracy: 0.5169
Epoch 4/30
496/496 [==============================] - 11s 22ms/step - loss: 1.3067 - accuracy: 0.5152 - val_loss: 1.3206 - val_accuracy: 0.5169
Epoch 5/30
496/496 [==============================] - 13s 26ms/step - loss: 1.3076 - accuracy: 0.5152 - val_loss: 1.3189 - val_accuracy: 0.5169
Epoch 6/30
496/496 [==============================] - 13s 27ms/step - loss: 1.3045 - accuracy: 0.5152 - val_loss: 1.3178 - val_accuracy: 0.5169
Epoch 7/30
496/496 [==============================] - 12s 23ms/step - loss: 1.3032 - accuracy: 0.5152 - val_loss: 1.3175 - val_accuracy:

496/496 [==============================] - 10s 20ms/step - loss: 1.1886 - accuracy: 0.5557 - val_loss: 1.5853 - val_accuracy: 0.4987
Epoch 28/30
496/496 [==============================] - 10s 20ms/step - loss: 1.1803 - accuracy: 0.5580 - val_loss: 1.6102 - val_accuracy: 0.5008
Epoch 29/30
496/496 [==============================] - 10s 20ms/step - loss: 1.1614 - accuracy: 0.5639 - val_loss: 1.7706 - val_accuracy: 0.4962
Epoch 30/30
496/496 [==============================] - 10s 21ms/step - loss: 1.1440 - accuracy: 0.5697 - val_loss: 1.7099 - val_accuracy: 0.5008
Epoch 1/30
496/496 [==============================] - 10s 20ms/step - loss: 1.1319 - accuracy: 0.5731 - val_loss: 1.6562 - val_accuracy: 0.4977
Epoch 2/30
496/496 [==============================] - 10s 20ms/step - loss: 1.1159 - accuracy: 0.5815 - val_loss: 1.8191 - val_accuracy: 0.4977
Epoch 3/30
496/496 [==============================] - 10s 20ms/step - loss: 1.1099 - accuracy: 0.5847 - val_loss: 1.7202 - val_accuracy: 0.4997


In [ ]:
from keras.preprocessing import image as keras_image

test_image = keras_image.load_img('train/00a1ae8867e0bb89f061679e1cf29e80.jpg', target_size = (image_size, image_size))
print(test_image)
plt.imshow(test_image)


In [ ]:
print("Mean value of train "+ str(train_pred.mean()))
print("Mean value of train "+ str(test_pred.mean()))


In [ ]:
display(train.head())

In [32]:
print(result[10])

[0.1812706  0.49880078 0.1896179  0.07615223 0.05415849]


In [34]:
#test_image = keras_image.img_to_array(test_image)
#test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(Y)
final=pd.DataFrame()
final['Id']=test['Id']
final['Subject Focus']=test['Subject Focus']
final['Eyes']=test['Eyes']
final['Face']=test['Face']
final['Near']=test['Near']
final['Action']=test['Action']
final['Accessory']=test['Accessory']
final['Group']=test['Group']
final['Collage']=test['Collage']
final['Human']=test['Human']
final['Occlusion']=test['Occlusion']
final['Info']=test['Info']
final['Blur']=test['Blur']
final['Pawpularity']=test['Pawpularity']
print(final)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
display(final.head())

In [ ]:
a = len(final['Pawpularity'])
b = final['Pawpularity']
rmse = train_pred.mean()
temp = []
temp2 = []

for number in range(a):
    temp.append(rmse)
for i in range(a):
    temp2.append(test_pred[i])


In [ ]:
import math
MSE = np.square(np.subtract(temp2,temp)).mean() 


In [ ]:
value = math.sqrt(MSE)
print(value)

In [ ]:
# list all data in history1
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# list all data in history2
print(history.history.keys())
# summarize history for accuracy
plt.plot(history2.history['root_mean_squared_error'])
plt.plot(history2.history['val_root_mean_squared_error'])
plt.title('model rmse')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# list all data in history3
print(history.history.keys())
# summarize history for accuracy
plt.plot(history3.history['root_mean_squared_error'])
plt.plot(history3.history['val_root_mean_squared_error'])
plt.title('model rmse')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()